<a href="https://colab.research.google.com/github/Kepners/ChopOnions/blob/main/Trndzo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# Block 1: Install Required Libraries
!pip install openai==0.27.8
!pip install python-dotenv
!pip install praw
!pip install requests
!pip install cachetools
!pip install nltk
!pip install tqdm
!pip install numpy
!pip install scikit-learn
!pip install feedparser
!pip install google-search-results  # SerpAPI client
!pip install pytrends  # Alternative to SerpAPI for Google Trends


In [30]:
# Block 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Block 3: Load Environment Variables
import os
from dotenv import load_dotenv
import logging

# Define the path to your .env file in Google Drive
dotenv_path = '/content/drive/MyDrive/Secrets/.env'  # Updated with capital 'S'

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

# Retrieve API keys from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PEXELS_API_KEY = os.getenv('PEXELS_API_KEY')
REDDIT_CLIENT_ID = os.getenv('REDDIT_CLIENT_ID')
REDDIT_CLIENT_SECRET = os.getenv('REDDIT_CLIENT_SECRET')
SHUTTERSTOCK_ACCESS_TOKEN = os.getenv('SHUTTERSTOCK_ACCESS_TOKEN')

# Validate that all required environment variables are set
required_vars = [
    'OPENAI_API_KEY',
    'PEXELS_API_KEY',
    'REDDIT_CLIENT_ID',
    'REDDIT_CLIENT_SECRET',
    'SHUTTERSTOCK_ACCESS_TOKEN'
]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    logging.error(f"Missing environment variables: {', '.join(missing_vars)}")
    raise SystemExit("Please ensure all API keys are set in the .env file.")
else:
    logging.info("All environment variables loaded successfully.")

# Block 4: Verify Environment Variables
for var in required_vars:
    if os.getenv(var):
        print(f"{var}: Loaded")
    else:
        print(f"{var}: Not Loaded")

# Block 5: Download NLTK Data
import nltk

# Download required NLTK data packages
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Block 6A: Create Utils Directory
!mkdir -p utils


# Block 6A: Create Utils Directory
import os

utils_dir = 'utils'
if not os.path.exists(utils_dir):
    os.makedirs(utils_dir)

# Block 6B: Create data_processing.py
data_processing_code = """
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def extract_keywords(text, max_keywords=10):
    \"""
    Extracts up to `max_keywords` nouns from the input `text`.
    \"""
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.isalpha() and word.lower() not in stop_words]
    # Get part-of-speech tags
    tagged_words = pos_tag(filtered_words)
    # Keep nouns and proper nouns
    keywords = [word for word, pos in tagged_words if pos.startswith('NN')]
    # Limit the number of keywords
    return keywords[:max_keywords]
"""

with open(os.path.join('utils', 'data_processing.py'), 'w') as file:
    file.write(data_processing_code)


# Block 7: Initialize API Clients and Configure Logging
import praw
import openai
import logging
import warnings
from utils.data_processing import extract_keywords

# Suppress PRAW warnings about asynchronous environments
warnings.filterwarnings("ignore", category=UserWarning, module='praw')

# Retrieve REDDIT_USER_AGENT from environment variables or set a default
REDDIT_USER_AGENT = os.getenv('REDDIT_USER_AGENT', 'script:TrendingTopicsScript:1.0 (by u/yourusername)')

# Initialize Reddit API using PRAW
try:
    reddit = praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secret=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT
    )
    logging.info("Reddit API initialized successfully.")
except Exception as e:
    logging.error(f"Error initializing Reddit API: {e}")

# Initialize OpenAI API
openai.api_key = OPENAI_API_KEY

# Configure Logging
logging.basicConfig(
    level=logging.ERROR,  # Set to ERROR to display only error messages
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()  # Log to console
    ]
)

# Function to add separators
def add_separator():
    print("")

# Define English, French, Spanish, Italian Speaking Countries
top_countries = {
    1: {'name': 'India', 'code': 'IN'},
    2: {'name': 'United States', 'code': 'US'},
    3: {'name': 'Nigeria', 'code': 'NG'},
    4: {'name': 'Mexico', 'code': 'MX'},
    5: {'name': 'Philippines', 'code': 'PH'},
    6: {'name': 'United Kingdom', 'code': 'GB'},
    7: {'name': 'France', 'code': 'FR'},
    8: {'name': 'Italy', 'code': 'IT'},
    9: {'name': 'South Africa', 'code': 'ZA'},
    10: {'name': 'Spain', 'code': 'ES'},  # Added Spain
    # Add more countries if needed
}

# Mapping from country codes to PyTrends 'pn' (payload name)
country_code_to_pn = {
    'IN': 'india',
    'US': 'united_states',
    'NG': 'nigeria',
    'MX': 'mexico',
    'PH': 'philippines',
    'GB': 'united_kingdom',
    'FR': 'france',
    'IT': 'italy',
    'ZA': 'south_africa',
    'ES': 'spain',  # Added Spain
    # Add more mappings as needed
}

# Function to Allow Users to Choose a Location for Google Trends
def select_country():
    print("Select a country for Google Trends data:")
    for idx, country in top_countries.items():
        print(f"{idx}. {country['name']}")

    # Get user selection
    try:
        country_selection = int(input("Enter the number of the country you're interested in: "))
        if country_selection in top_countries:
            selected_country = top_countries[country_selection]
            print(f"You selected: {selected_country['name']}")
        else:
            print("Invalid selection. Defaulting to United States.")
            selected_country = {'name': 'United States', 'code': 'US'}
    except ValueError:
        print("Invalid input. Defaulting to United States.")
        selected_country = {'name': 'United States', 'code': 'US'}

    return selected_country

# Function to Fetch Trending Topics from Google Trends using PyTrends
def fetch_trending_topics_pytrends(selected_country):
    add_separator()
    logging.info(f"Fetching trending topics for {selected_country['name']} from Google Trends via PyTrends...")
    pn = country_code_to_pn.get(selected_country['code'], 'united_states')  # Default to 'united_states' if not found
    try:
        pytrends = TrendReq(hl='en-US', tz=360)
        # Fetch daily trending searches
        trending_searches_df = pytrends.trending_searches(pn=pn)
        if trending_searches_df.empty:
            logging.error("No trending topics found using PyTrends.")
            return []
        trending_topics = trending_searches_df[0].tolist()
        trending_topics = trending_topics[:10]  # Limit to top 10 topics
        logging.info(f"Retrieved {len(trending_topics)} trending topics via PyTrends.")
        # Since PyTrends may not provide descriptions, we'll set them as empty
        trending_topics_with_desc = [{'title': topic, 'description': ''} for topic in trending_topics]
        return trending_topics_with_desc
    except Exception as e:
        logging.error(f"Error fetching Google Trends data via PyTrends: {e}")
        return []

# Function to Fetch Trending Topics from RSS Feeds (Alternative Approach)
def fetch_trending_topics_from_rss(feed_url, limit=10):
    add_separator()
    logging.info(f"Fetching trending topics from RSS feed: {feed_url}")
    try:
        feed = feedparser.parse(feed_url)
        trending_topics = []
        for entry in feed.entries[:limit]:
            title = entry.title
            description = entry.summary if 'summary' in entry else entry.title  # Use summary if available
            trending_topics.append({'title': title, 'description': description})
        logging.info(f"Retrieved {len(trending_topics)} trending topics from RSS.")
        return trending_topics
    except Exception as e:
        logging.error(f"Error fetching trending topics from RSS: {e}")
        return []

# Function to Generate a Two-Sentence Summary using OpenAI API
def generate_summary(content):
    add_separator()
    logging.info("Generating two-sentence summary using OpenAI API...")
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a concise summarizer. Provide a clear and brief two-sentence summary of the following content."
                    )
                },
                {
                    "role": "user",
                    "content": content
                }
            ],
            max_tokens=60,
            temperature=0.5,
        )
        summary = response['choices'][0]['message']['content'].strip()
        logging.info("Summary generated successfully.")
        return summary
    except Exception as e:
        logging.error(f"Error generating summary: {e}")
        return "No description available."

# Function to Search Subreddits Related to the Topic
def search_subreddits_for_topic(topic, limit=5):
    add_separator()
    logging.info(f"Searching for subreddits related to '{topic}'...")
    related_subreddits = []
    try:
        # Search for subreddits matching the topic
        for subreddit in reddit.subreddits.search(query=topic, limit=limit):
            related_subreddits.append({
                'name': subreddit.display_name,
                'title': subreddit.title,
                'description': subreddit.public_description or ''
            })
        logging.info(f"Found {len(related_subreddits)} subreddits related to '{topic}'.")
    except Exception as e:
        logging.error(f"Error searching for subreddits: {e}")
    return related_subreddits

# Function to Fetch Posts Related to a Topic from Reddit
def fetch_posts_from_reddit(topic, subreddit_name=None, limit=5):
    add_separator()
    logging.info(f"Fetching posts related to '{topic}' from Reddit...")
    posts = []
    try:
        if subreddit_name:
            subreddit = reddit.subreddit(subreddit_name)
            logging.info(f"Searching in subreddit: {subreddit_name}")
        else:
            subreddit = reddit.subreddit('all')
            logging.info("Searching in all of Reddit")
        # Search for the topic in Reddit
        for post in subreddit.search(topic, limit=limit):
            if not post.stickied and not post.over_18:
                posts.append({'title': post.title, 'score': post.score, 'url': post.url})
        logging.info(f"Retrieved {len(posts)} posts.")
    except Exception as e:
        logging.error(f"Error fetching posts related to '{topic}': {e}")

    return posts

# Keyword Extraction Function
def extract_keywords(text, max_keywords=5):
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.isalpha() and word.lower() not in stop_words]
    # Get part-of-speech tags
    tagged_words = pos_tag(filtered_words)
    # Keep nouns and proper nouns
    keywords = [word for word, pos in tagged_words if pos.startswith('NN')]
    # Limit the number of keywords
    return keywords[:max_keywords]

# Cache Setup and Function to Check Stock Media Availability
# Cache responses for 24 hours (86400 seconds)
cache = TTLCache(maxsize=1000, ttl=86400)

@cached(cache)
def check_stock_media_availability(topic):
    add_separator()
    logging.info(f"Checking stock media for topic: {topic}")
    # Extract keywords from the topic
    keywords = extract_keywords(topic)
    if not keywords:
        logging.error(f"No valid keywords extracted for topic: {topic}")
        return 0
    # Join keywords into a query string
    query = ' '.join(keywords)
    # Ensure the query does not exceed 100 characters
    query = query[:100]
    total_results = 0
    # Check Pexels
    total_results += check_pexels_media(query)
    # Check Shutterstock
    total_results += check_shutterstock_media(query)
    logging.info(f"Total stock media items available for '{topic}': {total_results}")
    return total_results

def check_pexels_media(query):
    headers = {
        'Authorization': PEXELS_API_KEY,
        'User-Agent': 'YourAppName'  # Optionally set User-Agent
    }
    params = {
        'query': query,
        'per_page': 1,
        'page': 1
    }
    try:
        response = requests.get('https://api.pexels.com/v1/search', headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            total_results = data.get('total_results', 0)
            return total_results
        else:
            logging.error(f"Pexels API error {response.status_code}: {response.text}")
            return 0
    except requests.exceptions.RequestException as e:
        logging.error(f"Error checking Pexels for '{query}': {e}")
        return 0

def check_shutterstock_media(query):
    headers = {
        'Authorization': f'Bearer {SHUTTERSTOCK_ACCESS_TOKEN}',
        'User-Agent': 'YourAppName'  # Optionally set User-Agent
    }
    params = {
        'query': query,
        'per_page': 1,
        'page': 1,
        'view': 'minimal'
    }
    try:
        response = requests.get('https://api.shutterstock.com/v2/images/search', headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            total_count = data.get('total_count', 0)
            return total_count
        else:
            logging.error(f"Shutterstock API error {response.status_code}: {response.text}")
            return 0
    except requests.exceptions.RequestException as e:
        logging.error(f"Error checking Shutterstock for '{query}': {e}")
        return 0

# Function to Generate Script for a Topic using OpenAI API
def generate_script_for_topic(topic, max_retries=5):
    add_separator()
    logging.info("Generating script using OpenAI API...")
    retry_delay = 1
    for attempt in range(max_retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {
                        "role": "system",
                        "content": (
                            "You are a creative scriptwriter for short-form videos. "
                            "Produce engaging, dynamic scripts that are visually compelling and can be represented "
                            "using stock images and videos. The script should have a captivating introduction, "
                            "a clear narrative flow, and a strong conclusion or call-to-action."
                        )
                    },
                    {
                        "role": "user",
                        "content": (
                            f"Write a short, captivating video script about '{topic}' suitable for platforms like TikTok. "
                            "Ensure the script includes vivid descriptions and is structured with a beginning, middle, and end. "
                            "Use language that resonates with the target audience, and make sure it's adaptable with widely available stock images and videos."
                        )
                    }
                ],
                max_tokens=600,
                temperature=0.7,
            )
            logging.info("Script generated successfully.")
            script_content = response['choices'][0]['message']['content'].strip()
            print(f"====\nGenerated script for '{topic}':\n{script_content}\n====\n")
            return script_content
        except openai.error.RateLimitError:
            logging.error(f"Rate limit hit. Waiting {retry_delay} seconds before retrying...")
            time.sleep(retry_delay)
            retry_delay *= 2
        except openai.error.OpenAIError as e:
            logging.error(f"An OpenAI error occurred: {e}")
            break
        except Exception as e:
            logging.error(f"An unexpected error occurred: {e}")
            break
    return "Max retries reached. Unable to generate script."

# Function to Fetch Subreddit Information (Optional)
def fetch_subreddit_info(limit=500):
    add_separator()
    logging.info("Fetching subreddit information...")
    subreddit_list = []
    try:
        # Get the top subreddits
        for subreddit in reddit.subreddits.popular(limit=limit):
            subreddit_list.append({
                'name': subreddit.display_name,
                'description': subreddit.public_description or ''
            })
        logging.info(f"Retrieved information for {len(subreddit_list)} subreddits.")
    except Exception as e:
        logging.error(f"Error fetching subreddits: {e}")
    return subreddit_list

# Function to Find Similar Subreddits (Optional)
def compute_embeddings(text_list):
    embeddings = []
    for text in tqdm(text_list, desc="Computing embeddings"):
        try:
            response = openai.Embedding.create(
                input=text,
                model="text-embedding-ada-002"
            )
            embeddings.append(response['data'][0]['embedding'])
        except Exception as e:
            logging.error(f"Error computing embedding for text: {e}")
            embeddings.append([0]*1536)  # Handle error by adding a zero vector
    return embeddings

def find_similar_subreddits(input_text, subreddit_list, top_n=10):
    add_separator()
    logging.info("Computing embedding for the input text...")
    try:
        input_embedding = openai.Embedding.create(
            input=input_text,
            model="text-embedding-ada-002"
        )['data'][0]['embedding']
    except Exception as e:
        logging.error(f"Error computing embedding for input text: {e}")
        return []

    logging.info("Computing embeddings for subreddit descriptions...")
    descriptions = [sub['description'] for sub in subreddit_list]
    embeddings = compute_embeddings(descriptions)

    logging.info("Calculating similarities...")
    similarities = cosine_similarity([input_embedding], embeddings)[0]

    # Combine subreddits with similarities
    for i, sub in enumerate(subreddit_list):
        sub['similarity'] = similarities[i]

    # Sort subreddits by similarity
    subreddit_list.sort(key=lambda x: x['similarity'], reverse=True)

    # Return top N similar subreddits
    similar_subreddits = subreddit_list[:top_n]
    logging.info(f"Found {len(similar_subreddits)} similar subreddits.")
    return similar_subreddits

# Main Execution Block
def main():
    add_separator()
    print("====")
    print("Starting script execution...\n")

    # Step 1: User selects the country for Google Trends
    selected_country = select_country()

    # Step 2: Fetch trending topics for the selected country
    # Attempt PyTrends first
    google_trends_topics = fetch_trending_topics_pytrends(selected_country)

    # If PyTrends fails or returns insufficient data, try RSS feeds
    if not google_trends_topics:
        logging.info("No trending topics found using PyTrends. Attempting to fetch from RSS feeds.")
        # Define RSS feed URLs based on country or default
        # Example RSS feed URL (you may need to replace this with actual RSS feeds related to Google Trends)
        rss_feed_url = f"https://trends.google.com/trends/trendingsearches/daily/rss?geo={selected_country['code']}"
        google_trends_topics = fetch_trending_topics_from_rss(rss_feed_url)

    if not google_trends_topics:
        logging.error("No trending topics found using all available methods.")
        return

    # Generate descriptions for trending topics if missing
    for topic in google_trends_topics:
        if not topic['description'] or topic['description'].lower() == "no description available.":
            topic['description'] = generate_summary(topic['title'])

    # Display the trending topics with descriptions to the user
    print(f"\n====\nCurrent Trending Topics in {selected_country['name']}:")
    for idx, topic in enumerate(google_trends_topics, start=1):
        title = topic['title']
        description = topic['description'] if topic['description'] else "No description available."
        print(f"{idx}. {title} - {description}")

    # Allow the user to select a topic
    try:
        selected_idx = int(input("\nEnter the number of the topic you're interested in: "))
        if 1 <= selected_idx <= len(google_trends_topics):
            selected_topic = google_trends_topics[selected_idx - 1]['title']
            print(f"\nYou selected: {selected_topic}")
        else:
            logging.error("Invalid selection. Exiting.")
            return
    except ValueError:
        logging.error("Invalid input. Please enter a number. Exiting.")
        return

    # Step 3: Search for subreddits related to the topic
    related_subreddits = search_subreddits_for_topic(selected_topic, limit=5)
    if not related_subreddits:
        logging.error("No related subreddits found. Exiting.")
        return

    # Display related subreddits to the user
    print("\n====\nRelated Subreddits:")
    for idx, sub in enumerate(related_subreddits, start=1):
        print(f"{idx}. {sub['name']} - {sub['title']}")

    # Fetch and display top two posts from each subreddit
    print("\n====\nTop 2 Posts from Each Subreddit:")
    all_posts = []
    for sub in related_subreddits:
        subreddit_name = sub['name']
        try:
            subreddit = reddit.subreddit(subreddit_name)
            top_posts = list(subreddit.hot(limit=2))
            print(f"\nSubreddit: {subreddit_name}")
            for post in top_posts:
                print(f"- {post.title} (Score: {post.score})")
                all_posts.append({'subreddit': subreddit_name, 'title': post.title, 'score': post.score, 'url': post.url})
        except Exception as e:
            logging.error(f"Error fetching posts from subreddit '{subreddit_name}': {e}")
            print(f"- Unable to fetch posts from subreddit '{subreddit_name}'.")

    if not all_posts:
        logging.error("No posts fetched from the related subreddits. Exiting.")
        return

    # Step 4: Allow user to select specific posts to generate scripts for
    print("\n====\nAll Fetched Posts:")
    for idx, post in enumerate(all_posts, start=1):
        print(f"{idx}. [{post['subreddit']}] {post['title']} (Score: {post['score']})")

    selected_posts_input = input("\nEnter the numbers of the posts you want to generate scripts for, separated by commas (e.g., 1,3,5): ")
    try:
        selected_indices = [int(i.strip()) - 1 for i in selected_posts_input.split(',') if i.strip().isdigit()]
        selected_posts = [all_posts[i] for i in selected_indices if 0 <= i < len(all_posts)]
        if not selected_posts:
            logging.error("No valid posts selected. Exiting.")
            return
    except ValueError:
        logging.error("Invalid input. Please enter numbers separated by commas. Exiting.")
        return

    # Fetch content and generate summaries for selected posts
    print(f"\n====\nFetching content and generating summaries for selected posts...")
    for post in selected_posts:
        title = post['title']
        url = post['url']
        try:
            reddit_post = reddit.submission(url=url)
            content = reddit_post.selftext if reddit_post.is_self else reddit_post.title
            summary = generate_summary(content)
            post['summary'] = summary
        except Exception as e:
            logging.error(f"Error fetching content for post '{title}': {e}")
            post['summary'] = "No summary available."

    # Step 5: Decide Whether to Check Stock Media Availability
    check_media = input("\nDo you want to check for stock media availability for the selected posts? (yes/no): ").strip().lower()
    if check_media not in ['yes', 'y']:
        add_separator()
        print("====")
        print("====")
        print("Skipping stock media availability check.")
        # Proceed to generate scripts without checking
        for post in selected_posts:
            title = post['title']
            summary = post.get('summary', "No summary available.")
            print(f"\nSummary: {summary}")
            script = generate_script_for_topic(title)
            print(f"====\nGenerated script for '{title}':\n{script}\n====\n")
        print("\nScript execution completed.")
        return

    # Continue with stock media availability check
    add_separator()
    print("====")
    print("Checking stock media availability for the selected posts...")
    request_count = 0
    scripts_generated = 0
    for post in selected_posts:
        title = post['title']
        summary = post.get('summary', "No summary available.")
        if request_count >= 190:
            logging.error("Approaching API rate limits. Waiting for 60 minutes before continuing...")
            print("====")
            print("====")
            time.sleep(3600)  # Wait for an hour
            request_count = 0
        total_media = check_stock_media_availability(title)
        request_count += 1
        if total_media >= 10:
            logging.info(f"Topic '{title}' has {total_media} stock media items available.")
            # Display summary before generating script
            print(f"\nSummary: {summary}")
            # Step 6: Generate script for this topic
            script = generate_script_for_topic(title)
            scripts_generated += 1
        else:
            logging.info(f"Not enough stock media for topic: {title}")
            print(f"\nSummary: {summary}")
            print(f"Not enough stock media for topic: {title}\n====\n")
    if scripts_generated == 0:
        logging.error("No topics with sufficient stock media were found.")
    else:
        logging.info(f"Generated {scripts_generated} script(s) based on available stock media.")
    print("\nScript execution completed.")

# Block 8: Run the Main Function
if __name__ == "__main__":
    main()




OPENAI_API_KEY: Loaded
PEXELS_API_KEY: Loaded
REDDIT_CLIENT_ID: Loaded
REDDIT_CLIENT_SECRET: Loaded
SHUTTERSTOCK_ACCESS_TOKEN: Loaded


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!



====
Starting script execution...

Select a country for Google Trends data:
1. India
2. United States
3. Nigeria
4. Mexico
5. Philippines
6. United Kingdom
7. France
8. Italy
9. South Africa
10. Spain
Enter the number of the country you're interested in: 6
You selected: United Kingdom












====
Current Trending Topics in United Kingdom:
1. Pharrell Williams - Pharrell Williams is a multi-talented musician, producer, and fashion designer known for his work in the music industry and collaborations with various artists. He has achieved success both as a solo artist and as part of the duo The Neptunes and the group N.E.R.D.
2. Paddington - "Paddington" is a British family film based on the beloved children's book character, a polite and adventurous bear who finds a new home in London. The movie follows Paddington as he navigates city life, makes friends, and gets caught up in a series of comedic mishaps.
3. England vs New Zealand - England and New Zealand are two cricket teams that

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




You selected: Paddington



...


Subreddit: moviescirclejerk
- My Own Private Idaho [1991] (Score: 45)
- I Saw the Devil (2010) (Score: 176)

Subreddit: london
- Weekly Q&A Megathread. Please post any questions about visiting, tourism, living, working, budgeting, housing here! (Score: 3)
- London Needs This Too (Score: 2544)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




Subreddit: movies
- Join us Friday 11/8 at 3:00 PM ET for a live AMA/Q&A with Scott Beck & Bryan Woods, co-directors of the upcoming A24 horror film 'Heretic' (starring Hugh Grant, Sophie Thatcher, and Chloe East). They also directed '65' and co-wrote 'A Quiet Place'. (Score: 10)
- Hi /r/movies! We are Mary Dauterman and Grace Glowicki, writer/director and lead actor of BOOGER, a gross comedy body horror cat film that will make you puke. You can watch it on VOD now. Ask Us Anything! (Score: 9)

Subreddit: brisbane
- It's the /r/brisbane random discussion thread. 03/11/2024 (Score: 0)
- /r/Brisbane evening discussion thread 02 Nov, 2024 (Score: 6)

====
All Fetched Posts:
1. [Paddington] if you love paddington, chances are that you'll love bluey (Score: 18)
2. [Paddington] How did Paddington get his passport? (Score: 12)
3. [moviescirclejerk] My Own Private Idaho [1991] (Score: 45)
4. [moviescirclejerk] I Saw the Devil (2010) (Score: 176)
5. [london] Weekly Q&A Megathread. Please post 

ERROR:root:Error fetching content for post 'if you love paddington, chances are that you'll love bluey': Invalid URL (subreddit, not submission): http://old.reddit.com/r/bluey



====
Fetching content and generating summaries for selected posts...

Do you want to check for stock media availability for the selected posts? (yes/no): n

====
====
Skipping stock media availability check.

Summary: No summary available.

====
Generated script for 'if you love paddington, chances are that you'll love bluey':
[Opening shot of Paddington Bear figurine on a desk]

Narrator: "Hey, Paddington fans! Looking for a new favorite? Meet Bluey!"

[Cut to quick, fun clips of Bluey characters playing]

Narrator: "If you love the charm and heartwarming adventures of Paddington, chances are you'll fall head over heels for Bluey!"

[Transition to images of Bluey merchandise and clips from the show]

Narrator: "Bluey is a lovable Blue Heeler pup who, just like Paddington, brings joy, laughter, and life lessons in every episode."

[Exciting montage of Bluey's family having fun together]

Narrator: "Join Bluey, Bingo, and their family on imaginative adventures that will warm your heart